In [4]:
bentoml_bundle_path = '/home/bentoml/bentoml/repository/Service/20200424104819_1D90A9'

In [2]:
from clipper_admin import ClipperConnection, DockerContainerManager
from bentoml.clipper import deploy_bentoml
cl = ClipperConnection(DockerContainerManager())

try:
    cl.start_clipper(cache_size=1)
except:
    cl.connect()

APP_NAME = bentoml_bundle_path.split('/')[-1].lower()
cl.register_application(APP_NAME, 'strings', 'default_pred', 10 * 1000 * 1000)

20-04-24:10:39:44 INFO     [__init__.py:118] No local BentoML config file found, using default configurations
20-04-24:10:39:45 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-04-24:10:39:47 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpur7as4p7.yml
20-04-24:10:39:47 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running
20-04-24:10:39:47 INFO     [clipper_admin.py:236] [default-cluster] Application 20200423233754_0df217 was successfully registered


In [3]:
clipper_model_name, clipper_model_version = deploy_bentoml(cl, bentoml_bundle_path, 'predict_clipper')

[2020-04-24 10:39:50,225] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-04-24 10:39:50,243] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.2,  but loading from BentoML version 0.5.2+173.ga4278d4.dirty
[2020-04-24 10:39:51,441] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image

20-04-24:10:45:11 INFO     [docker_container_manager.py:409] [default-cluster] Found 0 replicas for service-predict-clipper:20200423233754-0df217. Adding 1
20-04-24:10:45:12 INFO     [clipper_admin.py:724] [default-cluster] Successfully registered model service-predict-clipper:20200423233754-0df217
20-04-24:10:45:12 INFO     [clipper_admin.py:642] [default-cluster] Done deploying model service-predict-clipper:20200423233754-0df217.


[2020-04-24 10:45:12,096] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-04-24 10:45:12,107] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.2,  but loading from BentoML version 0.5.2+173.ga4278d4.dirty


In [5]:
try:
    cl.link_model_to_app(APP_NAME, clipper_model_name)
except:
    pass
addr = cl.get_query_addr()
server_url = f"http://{addr}/{APP_NAME}/predict"
print(clipper_model_name)
server_url

20-04-24:10:53:32 INFO     [clipper_admin.py:303] [default-cluster] Model service-predict-clipper is now linked to application 20200423233754_0df217


service-predict-clipper


'http://localhost:1337/20200423233754_0df217/predict'

In [6]:
import random
import json

def get_request_producer():

    url = server_url
    method = "POST"
    headers = {"content-type": "application/json"}
    pred_sentence =  "The acting was a bit lacking."

    def _gen_data(size=1):
        raw = pred_sentence
        noise = f" By User:{random.random()}"
        data = json.dumps(
               {"input": raw + noise}  # random offset to avoid cache
        )
        return url, method, headers, data

    return _gen_data

# test
get_request_producer()()

('http://localhost:1337/20200423233754_0df217/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"input": "The acting was a bit lacking. By User:0.05275012138078261"}')

In [10]:
%%time
import requests
gen = get_request_producer()
for i in range(1):
    url, me, h, d = gen(i+1)
    r = requests.request(me, url, headers=h, data=d).json()
    print(r)

{'query_id': 3, 'output': 'negative', 'default': False}
CPU times: user 548 µs, sys: 2.01 ms, total: 2.56 ms
Wall time: 466 ms


In [11]:
from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(), lambda: 1, timeout=10)
b.start_session(120, total_user=50, spawn_speed=2)

======= Session started! =======

╒═════════════╤═════════════╤══════════╤═════════════════╤═══════════════════╕
│ Reqs/Fail   │   Failure % │   Reqs/s │ Avg Resp Time   │   Client Health % │
╞═════════════╪═════════════╪══════════╪═════════════════╪═══════════════════╡
│ 0/0         │           0 │        0 │ None            │               100 │
╘═════════════╧═════════════╧══════════╧═════════════════╧═══════════════════╛

╒═════════════╤═════════════╤══════════╤═════════════════╤═══════════════════╕
│ Reqs/Fail   │   Failure % │   Reqs/s │   Avg Resp Time │   Client Health % │
╞═════════════╪═════════════╪══════════╪═════════════════╪═══════════════════╡
│ 1/0         │           0 │      0.5 │         0.42637 │               100 │
╘═════════════╧═════════════╧══════════╧═════════════════╧═══════════════════╛

╒═════════════╤═════════════╤══════════╤═════════════════╤═══════════════════╕
│ Reqs/Fail   │   Failure % │   Reqs/s │   Avg Resp Time │   Client Health % │
╞═════════════╪═